#### WFIT Algorithm Implementation (Schnaitter 2011)

In [5]:
%load_ext autoreload
%autoreload 2

import os, sys
import IPython
notebook_path = IPython.get_ipython().starting_dir
target_subdirectory_path = os.path.abspath(os.path.join(os.path.dirname(notebook_path), 'PostgreSQL'))
sys.path.append(target_subdirectory_path)

from pg_utils import *
from ssb_qgen_class import *
from wfit import *

import time
import random
import pickle


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# create an SSB query generator object
qg = QGEN()

In [3]:
query = qg.generate_query(2)
#print(query)
C = extract_query_indexes(query, include_cols=False)
print(f"Candidate indexes:")
for i, index in enumerate(C):
    print(f"{i}: {index}")  


Candidate indexes:
0: Index name: ix_lineorder_lo_orderdate, Key cols: ('lo_orderdate',), Include cols: (), Current OID: None
1: Index name: ix_lineorder_lo_discount, Key cols: ('lo_discount',), Include cols: (), Current OID: None
2: Index name: ix_lineorder_lo_quantity, Key cols: ('lo_quantity',), Include cols: (), Current OID: None
3: Index name: ix_lineorder_lo_orderdate_lo_discount, Key cols: ('lo_orderdate', 'lo_discount'), Include cols: (), Current OID: None
4: Index name: ix_lineorder_lo_orderdate_lo_quantity, Key cols: ('lo_orderdate', 'lo_quantity'), Include cols: (), Current OID: None
5: Index name: ix_lineorder_lo_discount_lo_orderdate, Key cols: ('lo_discount', 'lo_orderdate'), Include cols: (), Current OID: None
6: Index name: ix_lineorder_lo_discount_lo_quantity, Key cols: ('lo_discount', 'lo_quantity'), Include cols: (), Current OID: None
7: Index name: ix_lineorder_lo_quantity_lo_orderdate, Key cols: ('lo_quantity', 'lo_orderdate'), Include cols: (), Current OID: None
8

In [4]:
# drop all existing indexes
conn = create_connection()
drop_all_indexes(conn)
close_connection(conn)

# test IBG 
ibg = IBG(query, C)
ibg.print_ibg()

Number of candidate indexes: 19
Getting hypothetical sizes of candidate indexes...
Created root node with id: 0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18
Constructing IBG...
No index scans were explicitly noted in the query plan.
Number of levels in IBG: 7
Number of nodes in IBG: 16, Total number of what-if calls: 16, Time spent on what-if calls: 0.03925728797912598
Computing all pair degree of interaction...


Processing nodes in parallel: 100%|██████████| 4/4 [00:00<00:00, 58052.65it/s]

Time spent on computing all pair degree of interaction: 0.05771684646606445
0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18 -> 
0_1_2_3_4_5_6_7_8_10_11_12_13_14_15_16_17_18 -> 0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17 -> 
0_1_2_3_4_5_6_7_8_11_12_13_14_15_16_17_18 -> 0_1_2_3_4_5_6_7_8_10_11_12_13_14_15_16_17 -> 0_1_2_3_4_5_6_7_8_10_11_12_13_14_15_16_17 -> 0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_17 -> 
0_1_2_4_5_6_7_8_11_12_13_14_15_16_17_18 -> 0_1_2_3_4_5_6_7_8_11_12_13_14_15_16_17 -> 0_1_2_3_4_5_6_7_8_11_12_13_14_15_16_17 -> 0_1_2_3_4_5_6_7_8_10_11_12_13_14_15_17 -> 0_1_2_3_4_5_6_7_8_11_12_13_14_15_16_17 -> 0_1_2_3_4_5_6_7_8_10_11_12_13_14_15_17 -> 0_1_2_3_4_5_6_7_8_10_11_12_13_14_15_17 -> 0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15 -> 
0_1_2_4_5_6_7_8_11_12_13_14_15_16_17 -> 0_1_2_4_5_6_7_8_11_12_13_14_15_16_17 -> 0_1_2_3_4_5_6_7_8_11_12_13_14_15_17 -> 0_1_2_4_5_6_7_8_11_12_13_14_15_16_17 -> 0_1_2_3_4_5_6_7_8_11_12_13_14_15_17 -> 0_1_2_3_4_5_6_7_8_11_12_13_14_15_17 -> 0_1_2_3_4_5_6_7_8_10_11

In [6]:
# compute the max benefit of each candidate index and print the sorted list
max_benefits = [(index, ibg.compute_max_benefit(index)) for index in C]
max_benefits = sorted(max_benefits, key=lambda x: x[1], reverse=True)
print(f"Max benefits:")
for index, benefit in max_benefits:
    print(f"{index.index_id}: {benefit}")


Max benefits:
ix_lineorder_lo_orderdate_lo_discount_lo_quantity: 813408.31
ix_lineorder_lo_orderdate_lo_quantity_lo_discount: 795873.5100000001
ix_lineorder_lo_orderdate_lo_discount: 789687.56
ix_dwdate_d_yearmonthnum_d_datekey: 74.40000000002328
ix_dwdate_d_yearmonthnum: 70.40000000002328
ix_dwdate_d_datekey_d_yearmonthnum: 23.449999999953434
ix_lineorder_lo_orderdate: 0
ix_lineorder_lo_discount: 0
ix_lineorder_lo_quantity: 0
ix_lineorder_lo_orderdate_lo_quantity: 0
ix_lineorder_lo_discount_lo_orderdate: 0
ix_lineorder_lo_discount_lo_quantity: 0
ix_lineorder_lo_quantity_lo_orderdate: 0
ix_lineorder_lo_quantity_lo_discount: 0
ix_lineorder_lo_discount_lo_orderdate_lo_quantity: 0
ix_lineorder_lo_discount_lo_quantity_lo_orderdate: 0
ix_lineorder_lo_quantity_lo_orderdate_lo_discount: 0
ix_lineorder_lo_quantity_lo_discount_lo_orderdate: 0
ix_dwdate_d_datekey: 0


In [7]:
# pick random subset of candidate indexes
X = random.sample(ibg.C, 8)
cost, used = ibg.get_cost_used(X)
print(f"IBG     --> Cost: {cost}, Used indexes: {[idx.index_id for idx in used]}")

cost, used = ibg._cached_get_cost_used(X)
print(f"What-if --> Cost: {cost}, Used indexes: {[idx.index_id for idx in used]}")

# pick two indexes and a configuration
a = ibg.C[0]
b = ibg.C[4] 
X = [ibg.C[1], ibg.C[2], ibg.C[5], ibg.C[6], ibg.C[8]]

# compute maximum benefit of adding index 'a' 
max_benefit = ibg.compute_max_benefit(a)
print(f"\nMaximum benefit of adding index {a.index_id}: {max_benefit}")

# compute degree of interaction between indexes 'a' and 'b' in configuration X
doi = ibg.compute_doi_configuration(a, b, X)
print(f"\nDOI between indexes {a.index_id} and {b.index_id} : {doi}")
print(f"in configuration {[idx.index_id for idx in X]}")

# compute configuration independent degree of interaction between indexes 'a' and 'b'
doi = ibg.get_doi_pair(a, b)
print(f"\nDOI between indexes {a.index_id} and {b.index_id} : {doi}")


IBG     --> Cost: 712281.04, Used indexes: ['ix_lineorder_lo_orderdate_lo_discount', 'ix_dwdate_d_yearmonthnum']
What-if --> Cost: 712281.04, Used indexes: ['ix_lineorder_lo_orderdate_lo_discount', 'ix_dwdate_d_yearmonthnum']

Maximum benefit of adding index ix_lineorder_lo_orderdate: 0

DOI between indexes ix_lineorder_lo_orderdate and ix_lineorder_lo_orderdate_lo_quantity : 0.0
in configuration ['ix_lineorder_lo_discount', 'ix_lineorder_lo_quantity', 'ix_lineorder_lo_discount_lo_orderdate', 'ix_lineorder_lo_discount_lo_quantity', 'ix_lineorder_lo_quantity_lo_discount']

DOI between indexes ix_lineorder_lo_orderdate and ix_lineorder_lo_orderdate_lo_quantity : 0


In [8]:
#for key, value in ibg.doi.items():
#    print(f"doi({key[0]},   {key[1]}) = {value}")

#### Load workload from file

In [6]:
# load the workload from a file
with open('ssb_static_workload_4.pkl', 'rb') as f:
    workload_dict = pickle.load(f) 

workload_metadata = workload_dict['metadata']
workload = workload_dict['workload']    

print(f"Num rounds: {workload_metadata['num_rounds']}")
print(f"Template sequence: {workload_metadata['template_sequence']}")

Num rounds: 100
Template sequence: [1, 9, 4, 14, 2, 6, 8, 3, 5, 13, 7, 12]


In [11]:
"""# instatiate HypoGreedy object
hg = HypoGreedy(config_memory_MB=1024*12)

# run greedy algorithm on 100 queries
for i, query_object in enumerate(workload[:50]):
    print(f"\nProcessing query # {i+1}")
    hg.process_greedy(query_object)"""
    

'# instatiate HypoGreedy object\nhg = HypoGreedy(config_memory_MB=1024*12)\n\n# run greedy algorithm on 100 queries\nfor i, query_object in enumerate(workload[:50]):\n    print(f"\nProcessing query # {i+1}")\n    hg.process_greedy(query_object)'

#### Test WFIT implementation on sample SSB workload

In [13]:
"""# generate an SSB workload
workload = [qg.generate_query(i) for i in ([1,2,3]*10)]

print(len(workload))"""

'# generate an SSB workload\nworkload = [qg.generate_query(i) for i in ([1,2,3]*10)]\n\nprint(len(workload))'

In [14]:
# Workload 3 Experiment --> `wfit = WFIT(S_0, max_key_columns=3, include_cols=False, max_U=None, idxCnt=25, stateCnt=500, rand_cnt=100, creation_cost_fudge_factor=2048)`

In [15]:
# instantiate WFIT
#C = extract_query_indexes(qg.generate_query(8), include_cols=False)  
S_0 = []#C[0:1]
wfit = WFIT(S_0, max_key_columns=3, include_cols=False, max_U=None, ibg_max_nodes=1000, doi_max_nodes=500, max_doi_iters_per_node=1000, idxCnt=50, stateCnt=500, rand_cnt=100, creation_cost_fudge_factor=1024)

# process the workload
for i, query in enumerate(workload[:24]):
    print(f"Processing query {i+1}")
    print('-----------------------------------')
    wfit.process_WFIT(query, remove_stale_U=False, remove_stale_freq=1, execute=True, materialize=True, verbose=True)
    print("\n\n")

##################################################################
Initializing WFA instances for 1 stable partitions...
WFA instance #0: {(): 0}
Initial set of materialized indexes: []
Stable partitions: [[]]
Initial work function instances: 
	WFA Instance #0: {(): 0}

Maximum number of candidate indexes tracked: 50
Maximum number of MTS states/configurations: 500
Maximum number of historical index statistics kept: 100
Number of randomized clustering iterations: 100
*** Dropping all materialized indexes...
##################################################################

Processing query 1
-----------------------------------
Generating new partitions for query #1
Extracted 19 new indexes from query.
Candidate indexes (including those currently materialized), |U| = 19
Computing IBG...
Number of candidate indexes: 19
Getting hypothetical sizes of candidate indexes...
Created root node with id: 0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18
Constructing IBG...
Number of levels in IBG: 

Processing nodes in parallel: 100%|██████████| 5/5 [00:00<00:00, 54330.36it/s]


Time spent on computing all pair degree of interaction: 0.12256550788879395
Updating statistics...
Choosing top 50 indexes from 19 non-materialized candidate indexes
19 top indexes: ['ix_lineorder_lo_quantity', 'ix_dwdate_d_year', 'ix_dwdate_d_datekey_d_year', 'ix_dwdate_d_year_d_datekey', 'ix_dwdate_d_datekey', 'ix_lineorder_lo_quantity_lo_orderdate', 'ix_lineorder_lo_quantity_lo_discount', 'ix_lineorder_lo_orderdate_lo_quantity', 'ix_lineorder_lo_quantity_lo_orderdate_lo_discount', 'ix_lineorder_lo_quantity_lo_discount_lo_orderdate', 'ix_lineorder_lo_orderdate_lo_quantity_lo_discount', 'ix_lineorder_lo_orderdate_lo_discount_lo_quantity', 'ix_lineorder_lo_discount_lo_quantity', 'ix_lineorder_lo_discount_lo_orderdate_lo_quantity', 'ix_lineorder_lo_discount_lo_quantity_lo_orderdate', 'ix_lineorder_lo_orderdate', 'ix_lineorder_lo_discount', 'ix_lineorder_lo_orderdate_lo_discount', 'ix_lineorder_lo_discount_lo_orderdate']
New set of indexes to monitor for upcoming workload, |D| = 19
Choos

Processing nodes in parallel: 100%|██████████| 4/4 [00:11<00:00,  2.97s/it]


Time spent on computing all pair degree of interaction: 12.171681642532349
Updating statistics...
Choosing top 49 indexes from 40 non-materialized candidate indexes
40 top indexes: ['ix_lineorder_lo_quantity_lo_orderdate', 'ix_lineorder_lo_quantity_lo_discount', 'ix_lineorder_lo_quantity_lo_orderdate_lo_discount', 'ix_lineorder_lo_quantity_lo_discount_lo_orderdate', 'ix_lineorder_lo_orderdate_lo_quantity', 'ix_lineorder_lo_orderdate_lo_quantity_lo_discount', 'ix_lineorder_lo_orderdate_lo_discount_lo_quantity', 'ix_lineorder_lo_discount_lo_quantity', 'ix_lineorder_lo_discount_lo_orderdate_lo_quantity', 'ix_lineorder_lo_discount_lo_quantity_lo_orderdate', 'ix_supplier_s_suppkey_s_city', 'ix_supplier_s_city_s_suppkey', 'ix_supplier_s_city', 'ix_dwdate_d_year_d_datekey', 'ix_dwdate_d_year', 'ix_dwdate_d_datekey_d_year', 'ix_dwdate_d_datekey', 'ix_lineorder_lo_orderdate', 'ix_lineorder_lo_discount', 'ix_lineorder_lo_orderdate_lo_discount', 'ix_lineorder_lo_discount_lo_orderdate', 'ix_suppli

...

Time spent on computing all pair degree of interaction: 36.424986124038696
Updating statistics...
Choosing top 50 indexes from 145 non-materialized candidate indexes
50 top indexes: ['ix_lineorder_lo_orderdate_lo_quantity_lo_discount', 'ix_lineorder_lo_orderdate_lo_discount_lo_quantity', 'ix_lineorder_lo_orderdate_lo_quantity', 'ix_lineorder_lo_orderdate_lo_discount', 'ix_lineorder_lo_quantity_lo_linenumber', 'ix_lineorder_lo_orderdate_lo_suppkey', 'ix_lineorder_lo_orderdate_lo_suppkey_lo_custkey', 'ix_lineorder_lo_partkey', 'ix_lineorder_lo_partkey_lo_orderdate', 'ix_lineorder_lo_partkey_lo_suppkey', 'ix_lineorder_lo_partkey_lo_orderdate_lo_suppkey', 'ix_lineorder_lo_partkey_lo_suppkey_lo_orderdate', 'ix_lineorder_lo_orderdate', 'ix_lineorder_lo_orderdate_lo_custkey', 'ix_lineorder_lo_orderdate_lo_partkey', 'ix_lineorder_lo_orderdate_lo_custkey_lo_suppkey', 'ix_lineorder_lo_quantity', 'ix_lineorder_lo_quantity_lo_orderdate', 'ix_lineorder_lo_quantity_lo_discount', 'ix_lineorder_lo_qua

Processing nodes in parallel: 100%|██████████| 4/4 [00:02<00:00,  1.49it/s]


Time spent on computing all pair degree of interaction: 2.9168577194213867
Updating statistics...
Choosing top 50 indexes from 145 non-materialized candidate indexes
50 top indexes: ['ix_lineorder_lo_orderdate_lo_quantity_lo_discount', 'ix_lineorder_lo_orderdate_lo_discount_lo_quantity', 'ix_lineorder_lo_orderdate_lo_quantity', 'ix_lineorder_lo_orderdate_lo_discount', 'ix_lineorder_lo_quantity_lo_linenumber', 'ix_lineorder_lo_orderdate_lo_suppkey', 'ix_lineorder_lo_orderdate_lo_suppkey_lo_custkey', 'ix_lineorder_lo_partkey', 'ix_lineorder_lo_partkey_lo_orderdate', 'ix_lineorder_lo_partkey_lo_suppkey', 'ix_lineorder_lo_partkey_lo_orderdate_lo_suppkey', 'ix_lineorder_lo_partkey_lo_suppkey_lo_orderdate', 'ix_lineorder_lo_orderdate', 'ix_lineorder_lo_orderdate_lo_custkey', 'ix_lineorder_lo_orderdate_lo_partkey', 'ix_lineorder_lo_orderdate_lo_custkey_lo_suppkey', 'ix_lineorder_lo_quantity', 'ix_lineorder_lo_quantity_lo_orderdate', 'ix_lineorder_lo_quantity_lo_discount', 'ix_lineorder_lo_qua

Processing nodes in parallel: 100%|██████████| 4/4 [00:04<00:00,  1.08s/it]


Time spent on computing all pair degree of interaction: 4.591541290283203
Updating statistics...
Choosing top 50 indexes from 145 non-materialized candidate indexes
50 top indexes: ['ix_lineorder_lo_orderdate_lo_quantity_lo_discount', 'ix_lineorder_lo_orderdate_lo_discount_lo_quantity', 'ix_lineorder_lo_orderdate_lo_quantity', 'ix_lineorder_lo_orderdate_lo_discount', 'ix_lineorder_lo_quantity_lo_linenumber', 'ix_lineorder_lo_orderdate_lo_suppkey', 'ix_lineorder_lo_orderdate_lo_suppkey_lo_custkey', 'ix_lineorder_lo_partkey', 'ix_lineorder_lo_partkey_lo_orderdate', 'ix_lineorder_lo_partkey_lo_suppkey', 'ix_lineorder_lo_partkey_lo_orderdate_lo_suppkey', 'ix_lineorder_lo_partkey_lo_suppkey_lo_orderdate', 'ix_lineorder_lo_orderdate', 'ix_lineorder_lo_orderdate_lo_custkey', 'ix_lineorder_lo_orderdate_lo_partkey', 'ix_lineorder_lo_orderdate_lo_custkey_lo_suppkey', 'ix_lineorder_lo_quantity', 'ix_lineorder_lo_quantity_lo_orderdate', 'ix_lineorder_lo_quantity_lo_discount', 'ix_lineorder_lo_quan

In [51]:
wfit.execution_time

[4.940416,
 4.18169,
 5.005723,
 3.7669810000000004,
 5.287907000000001,
 4.4011700000000005,
 5.238727,
 5.474468,
 7.719691,
 4.810462,
 5.2755730000000005,
 5.933515000000001,
 5.134351,
 4.5287820000000005,
 5.311734,
 4.478656,
 5.401604,
 4.779478,
 4.874202,
 5.507383,
 5.7621139999999995,
 4.993757,
 5.079637,
 6.166906,
 5.066616,
 4.4437120000000006,
 5.7895069999999995,
 4.6986419999999995,
 5.168814,
 5.806243,
 4.69567,
 3.285181,
 8.343732,
 5.720944,
 5.664234,
 6.160909999999999,
 5.537100000000001,
 4.923020999999999,
 5.697332,
 4.2147060000000005,
 7.500965,
 5.189662,
 5.3942820000000005,
 4.085491,
 6.904523,
 5.598812,
 6.1531210000000005,
 6.025687,
 5.992545,
 4.7241670000000004,
 5.783834,
 4.197801999999999,
 5.9609570000000005,
 4.804457,
 4.683084,
 5.450734000000001,
 4.84058,
 5.283640999999999,
 4.91287,
 6.231037000000001]

In [7]:
# execute workload without any indexes
total_time_noIndex = execute_workload_noIndex(workload[:60], drop_indexes=True)

*** Dropping all existing indexes...
Index 'ix_dwdate_d_year' on table 'dwdate' dropped successfully
Index 'ix_lineorder_lo_orderdate_lo_quantity_lo_discount' on table 'lineorder' dropped successfully
Index 'ix_dwdate_d_year_d_datekey' on table 'dwdate' dropped successfully
Index 'ix_lineorder_lo_orderdate_lo_discount' on table 'lineorder' dropped successfully
Executing workload without any indexes...
PostgreSQL restarted successfully.
	Execution time for query# 1: 5.375473 s, Total time so far: 5.375473 s
PostgreSQL restarted successfully.
	Execution time for query# 2: 4.573349 s, Total time so far: 9.948822 s
PostgreSQL restarted successfully.
	Execution time for query# 3: 5.6781880000000005 s, Total time so far: 15.62701 s
PostgreSQL restarted successfully.
	Execution time for query# 4: 4.137888 s, Total time so far: 19.764898000000002 s
PostgreSQL restarted successfully.
	Execution time for query# 5: 5.679839 s, Total time so far: 25.444737000000003 s
PostgreSQL restarted successfu